In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

C:\Users\USER\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :',len(documents))

샘플의 수 : 11314


In [3]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [4]:
dataset.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [5]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [6]:
news_df = pd.DataFrame({'document':documents})

In [7]:
news_df.head(10)

,document
0,Well i'm not sure about the story nad it did s...
1,"\n\n\n\n\n\n\nYeah, do you expect people to re..."
2,Although I realize that principle is not one o...
3,Notwithstanding all the legitimate fuss about ...
4,"Well, I will have to change the scoring on my ..."
5,"\n \nI read somewhere, I think in Morton Smit..."
6,\nOk. I have a record that shows a IIsi with ...
7,\n\n\nSounds like wishful guessing.\n\n\n\n\n'...
8,Nobody is saying that you shouldn't be allowe...
9,\n I was wondering if anyone can shed any lig...


In [8]:

# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [9]:
news_df['clean_doc'][1]

"yeah, expect people read faq, etc. actually accept hard atheism? need little leap faith, jimmy. your logic runs steam! jim, sorry can't pity you, jim. sorry that have these feelings denial about faith need well, just pretend that will happily ever after anyway. maybe start newsgroup, alt.atheist.hard, won't bummin' much? bye-bye, jim. don't forget your flintstone's chewables! bake timmons,"

In [10]:

stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어 제거

In [11]:
print(tokenized_doc[1])

['yeah,', 'expect', 'people', 'read', 'faq,', 'etc.', 'actually', 'accept', 'hard', 'atheism?', 'need', 'little', 'leap', 'faith,', 'jimmy.', 'logic', 'runs', 'steam!', 'jim,', 'sorry', "can't", 'pity', 'you,', 'jim.', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well,', 'pretend', 'happily', 'ever', 'anyway.', 'maybe', 'start', 'newsgroup,', 'alt.atheist.hard,', "bummin'", 'much?', 'bye-bye,', 'jim.', 'forget', "flintstone's", 'chewables!', 'bake', 'timmons,']


In [12]:
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

TF-IDF 행렬의 크기 : (11314, 1000)


In [13]:
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

20

In [14]:
terms = vectorizer.get_feature_names_out() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('just', 0.20273), ('don', 0.19949), ('like', 0.19534), ('know', 0.18819), ('people', 0.17851)]
Topic 2: [('thanks', 0.31973), ('windows', 0.27738), ('card', 0.17369), ('drive', 0.15927), ('mail', 0.1488)]
Topic 3: [('game', 0.31991), ('team', 0.27972), ('year', 0.26585), ('games', 0.20684), ('drive', 0.17141)]
Topic 4: [('edu', 0.42972), ('thanks', 0.24949), ('mail', 0.17243), ('game', 0.12812), ('team', 0.12645)]
Topic 5: [('know', 0.41918), ('does', 0.305), ('thanks', 0.26133), ('don', 0.21061), ('just', 0.19501)]
Topic 6: [('drive', 0.45668), ('edu', 0.21787), ('thanks', 0.18804), ('scsi', 0.1573), ('drives', 0.12178)]
Topic 7: [('just', 0.56807), ('edu', 0.43053), ('don', 0.22492), ('like', 0.19983), ('soon', 0.09527)]
Topic 8: [('chip', 0.2163), ('government', 0.2001), ('encryption', 0.14658), ('like', 0.14639), ('clipper', 0.14265)]
Topic 9: [('don', 0.32188), ('know', 0.31942), ('edu', 0.28677), ('does', 0.26522), ('think', 0.19896)]
Topic 10: [('does', 0.47651), ('ca

In [15]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased., disagree, s...
1    [yeah,, expect, people, read, faq,, etc., actu...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal,,...
4    [well,, change, scoring, playoff, pool., unfor...
Name: clean_doc, dtype: object